# Magic State Injection Circuit for [[7,1,3]] Color Code

This implements the exact circuit from the figure that creates the logical observable qubit |ψ_L(θ,φ)⟩

In [14]:
from bloqade import squin
import bloqade.tsim

In [ ]:
# Magic state injection circuit for [[7,1,3]] color code
# Exact gate sequence from the specification:
# T1: √Y† on q[0–5]
# T2: CNOT(1→{2,3,4,5})
# T3: CNOT(0→{2,4})
# T4: √Y on q[2,3,4,5]
# T5: CNOT(2→3), CNOT(4→5)
# T6: CNOT(6→{1,3,5})
# T7: √Y on q[1,2,4]

@squin.kernel
def logical_qubit(theta: float = 0.0, phi: float = 0.0):
    # Allocate 7 qubits
    q = squin.qalloc(7)
    
    # T1: √Y† on q[0–5]
    # √Y† = RY(-π/2) = -1.5707963267948966
    squin.ry(angle=-1.5707963267948966, qubit=q[0])
    squin.ry(angle=-1.5707963267948966, qubit=q[1])
    squin.ry(angle=-1.5707963267948966, qubit=q[2])
    squin.ry(angle=-1.5707963267948966, qubit=q[3])
    squin.ry(angle=-1.5707963267948966, qubit=q[4])
    squin.ry(angle=-1.5707963267948966, qubit=q[5])
    
    # T2: CZ(1→2), CZ(3→4), CZ(5→6)
    squin.cz(control=q[1], target=q[2])
    squin.cz(control=q[3], target=q[4])
    squin.cz(control=q[5], target=q[6])

    
    # T3: √Y on q[6]
    # √Y = RY(π/2) = 1.5707963267948966
    squin.ry(angle=1.5707963267948966, qubit=q[6])
    
    # T4: CZ(0→3), CZ(2→5), CZ(4→6)
    squin.cz(control=q[0], target=q[3])
    squin.cz(control=q[2], target=q[5])
    squin.cz(control=q[4], target=q[6])
    
    # T5: √Y on q[2,3,4,5,6]
    squin.ry(angle=1.5707963267948966, qubit=q[2])
    squin.ry(angle=1.5707963267948966, qubit=q[3])
    squin.ry(angle=1.5707963267948966, qubit=q[4])
    squin.ry(angle=1.5707963267948966, qubit=q[5])
    squin.ry(angle=1.5707963267948966, qubit=q[6])

    # T6: CZ(0→1), CZ(2→3), CZ(4→5)
    squin.cz(control=q[0], target=q[1])
    squin.cz(control=q[2], target=q[3])
    squin.cz(control=q[4], target=q[5])
    
    # T7: √Y on q[1,2,4]
    squin.ry(angle=1.5707963267948966, qubit=q[1])
    squin.ry(angle=1.5707963267948966, qubit=q[2])
    squin.ry(angle=1.5707963267948966, qubit=q[4])

# Two copies of logical qubit on 12 qubits total
# First copy: qubits 0-6 (7 qubits)
# Second copy: qubits 7-11 (5 qubits, but we'll use 6-11 to make it 12 total)
# Actually, let's do: First copy 0-6, Second copy 5-11 (overlapping by 2 qubits) = 12 total
# Or better: First copy 0-6, Second copy 6-12 (but that's 13 qubits)
# Let's do: First copy on qubits 0-6, Second copy on qubits 5-11 (overlap at 5,6) = 12 qubits total



In [38]:
# Run 1000 shots on two_logical_qubits (from cell above)
from random import sample
import bloqade.stim

stim_circ = bloqade.stim.Circuit(two_logical_qubits)
sampler = stim_circ.compile_sampler()
samples = sampler.sample(shots=1000)

print((-1)**samples[0])
print((-1)**samples[1])
print((-1)**samples[2])


[ 1  1 -1 -1  1  1 -1]
[ 1  1 -1  1  1 -1 -1]
[ 1 -1  1 -1 -1  1  1]


In [36]:
print(f"Samples shape: {samples.shape} (1000 shots, {samples.shape[1]} measurements)")
print(f"Measured qubits: q7–q13 (second register)")
print(f"First 5 outcomes: {[''.join(map(str, s.astype(int))) for s in samples[:5]]}")

print((-1)**samples[0])

Samples shape: (1000, 7) (1000 shots, 7 measurements)
Measured qubits: q7–q13 (second register)
First 5 outcomes: ['1100100', '0010110', '1110000', '0100111', '0010000']
[-1 -1  1  1 -1  1  1]


In [65]:
# Create fixed-parameter version for Tsim visualization
# Three copies of the logical qubit circuit: q[0-6], q[7-13], q[14-20]
# Gates are applied in parallel (same timestep for all three logical qubits)
@squin.kernel
def _fixed():
    q = squin.qalloc(14)  # 3 registers × 7 qubits = 21 qubits
    
    # T1: √Y† on q[0–5], q[7–12], and q[14–19] (parallel)
    squin.ry(angle=-1.5707963267948966, qubit=q[0])  # √Y† = RY(-π/2)
    squin.ry(angle=-1.5707963267948966, qubit=q[1])
    squin.ry(angle=-1.5707963267948966, qubit=q[2])
    squin.ry(angle=-1.5707963267948966, qubit=q[3])
    squin.ry(angle=-1.5707963267948966, qubit=q[4])
    squin.ry(angle=-1.5707963267948966, qubit=q[5])
    squin.ry(angle=-1.5707963267948966, qubit=q[7])
    squin.ry(angle=-1.5707963267948966, qubit=q[8])
    squin.ry(angle=-1.5707963267948966, qubit=q[9])
    squin.ry(angle=-1.5707963267948966, qubit=q[10])
    squin.ry(angle=-1.5707963267948966, qubit=q[11])
    squin.ry(angle=-1.5707963267948966, qubit=q[12])
    squin.h(q[13])
    # T2: CZ gates (parallel for all three logical qubits)
    squin.cz(control=q[1], target=q[2])
    squin.cz(control=q[3], target=q[4])
    squin.cz(control=q[5], target=q[6])
    squin.cz(control=q[8], target=q[9])
    squin.cz(control=q[10], target=q[11])
    squin.cz(control=q[12], target=q[13])
    
    # T3: √Y on q[6], q[13], and q[20] (parallel)
    squin.ry(angle=1.5707963267948966, qubit=q[6])
    squin.ry(angle=1.5707963267948966, qubit=q[13])
    
    # T4: CZ gates (parallel for all three logical qubits)
    squin.cz(control=q[0], target=q[3])
    squin.cz(control=q[2], target=q[5])
    squin.cz(control=q[4], target=q[6])
    squin.cz(control=q[7], target=q[10])
    squin.cz(control=q[9], target=q[12])
    squin.cz(control=q[11], target=q[13])
    
    # T5: √Y on q[2,3,4,5,6], q[9,10,11,12,13], and q[16,17,18,19,20] (parallel)
    squin.ry(angle=1.5707963267948966, qubit=q[2])
    squin.ry(angle=1.5707963267948966, qubit=q[3])
    squin.ry(angle=1.5707963267948966, qubit=q[4])
    squin.ry(angle=1.5707963267948966, qubit=q[5])
    squin.ry(angle=1.5707963267948966, qubit=q[6])
    squin.ry(angle=1.5707963267948966, qubit=q[9])
    squin.ry(angle=1.5707963267948966, qubit=q[10])
    squin.ry(angle=1.5707963267948966, qubit=q[11])
    squin.ry(angle=1.5707963267948966, qubit=q[12])
    squin.ry(angle=1.5707963267948966, qubit=q[13])
    
    # T6: CZ gates (parallel for all three logical qubits)
    squin.cz(control=q[0], target=q[1])
    squin.cz(control=q[2], target=q[3])
    squin.cz(control=q[4], target=q[5])
    squin.cz(control=q[7], target=q[8])
    squin.cz(control=q[9], target=q[10])
    squin.cz(control=q[11], target=q[12])
    
    # T7: √Y on q[1,2,4], q[8,9,11], and q[15,16,18] (parallel)
    squin.ry(angle=1.5707963267948966, qubit=q[1])
    squin.ry(angle=1.5707963267948966, qubit=q[2])
    squin.ry(angle=1.5707963267948966, qubit=q[4])
    squin.ry(angle=1.5707963267948966, qubit=q[8])
    squin.ry(angle=1.5707963267948966, qubit=q[9])
    squin.ry(angle=1.5707963267948966, qubit=q[11])


    squin.cx(control=q[0], target=q[7])
    squin.cx(control=q[1], target=q[8])
    squin.cx(control=q[2], target=q[9])
    squin.cx(control=q[3], target=q[10])
    squin.cx(control=q[4], target=q[11])
    squin.cx(control=q[5], target=q[12])
    squin.cx(control=q[6], target=q[13])

    squin.measure(q[7])
    squin.measure(q[8])
    squin.measure(q[9])
    squin.measure(q[10])
    squin.measure(q[11])
    squin.measure(q[12])
    squin.measure(q[13])

    for i in range(7, 14):
        squin.reset(q[i])

# Visualize with white background
from IPython.display import HTML, display
tsim_circ = bloqade.tsim.Circuit(_fixed)
diagram = tsim_circ.diagram(height=500)
# Get HTML and wrap with white background
html_str = diagram._repr_html_() if hasattr(diagram, '_repr_html_') else str(diagram)
display(HTML(f'<div style="background-color: white; padding: 20px; border: 1px solid #ddd; display: inline-block;">{html_str}</div>'))

In [ ]:
from random import sample
import bloqade.stim

shot_length = 1000
stim_circ = bloqade.stim.Circuit(_fixed)
sampler = stim_circ.compile_sampler()
samples = sampler.sample(shots=shot_length)

def green_blue_orange_stabilizers(sample):
    z = (-1)**sample
    green = z[1] * z[2] * z[4] * z[5]
    blue = z[0] * z[1] * z[2] * z[3]
    orange = z[2] * z[3] * z[4] * z[6]
    return [green, blue, orange]

print((-1)**samples[0])
print((-1)**samples[1])
print((-1)**samples[2])

print(green_blue_orange_stabilizers(samples[0]))
print(green_blue_orange_stabilizers(samples[1]))
print(green_blue_orange_stabilizers(samples[2]))

first_stabilizer = green_blue_orange_stabilizers(samples[0])
    
for i, sample in enumerate(samples): 
    stabilizer = green_blue_orange_stabilizers(sample)
    if stabilizer != first_stabilizer:
        print(f"error at {i}")

len(samples)

In [64]:
# Create fixed-parameter version for Tsim visualization
# Three copies of the logical qubit circuit: q[0-6], q[7-13], q[14-20]
# Gates are applied in parallel (same timestep for all three logical qubits)
@squin.kernel
def _fixed():
    q = squin.qalloc(14)  # 3 registers × 7 qubits = 21 qubits
    
    # T1: √Y† on q[0–5], q[7–12], and q[14–19] (parallel)
    squin.ry(angle=-1.5707963267948966, qubit=q[0])  # √Y† = RY(-π/2)
    squin.ry(angle=-1.5707963267948966, qubit=q[1])
    squin.ry(angle=-1.5707963267948966, qubit=q[2])
    squin.ry(angle=-1.5707963267948966, qubit=q[3])
    squin.ry(angle=-1.5707963267948966, qubit=q[4])
    squin.ry(angle=-1.5707963267948966, qubit=q[5])
    squin.ry(angle=-1.5707963267948966, qubit=q[7])
    squin.ry(angle=-1.5707963267948966, qubit=q[8])
    squin.ry(angle=-1.5707963267948966, qubit=q[9])
    squin.ry(angle=-1.5707963267948966, qubit=q[10])
    squin.ry(angle=-1.5707963267948966, qubit=q[11])
    squin.ry(angle=-1.5707963267948966, qubit=q[12])
    squin.h(q[13])
    # T2: CZ gates (parallel for all three logical qubits)
    squin.cz(control=q[1], target=q[2])
    squin.cz(control=q[3], target=q[4])
    squin.cz(control=q[5], target=q[6])
    squin.cz(control=q[8], target=q[9])
    squin.cz(control=q[10], target=q[11])
    squin.cz(control=q[12], target=q[13])
    
    # T3: √Y on q[6], q[13], and q[20] (parallel)
    squin.ry(angle=1.5707963267948966, qubit=q[6])
    squin.ry(angle=1.5707963267948966, qubit=q[13])
    
    # T4: CZ gates (parallel for all three logical qubits)
    squin.cz(control=q[0], target=q[3])
    squin.cz(control=q[2], target=q[5])
    squin.cz(control=q[4], target=q[6])
    squin.cz(control=q[7], target=q[10])
    squin.cz(control=q[9], target=q[12])
    squin.cz(control=q[11], target=q[13])
    
    # T5: √Y on q[2,3,4,5,6], q[9,10,11,12,13], and q[16,17,18,19,20] (parallel)
    squin.ry(angle=1.5707963267948966, qubit=q[2])
    squin.ry(angle=1.5707963267948966, qubit=q[3])
    squin.ry(angle=1.5707963267948966, qubit=q[4])
    squin.ry(angle=1.5707963267948966, qubit=q[5])
    squin.ry(angle=1.5707963267948966, qubit=q[6])
    squin.ry(angle=1.5707963267948966, qubit=q[9])
    squin.ry(angle=1.5707963267948966, qubit=q[10])
    squin.ry(angle=1.5707963267948966, qubit=q[11])
    squin.ry(angle=1.5707963267948966, qubit=q[12])
    squin.ry(angle=1.5707963267948966, qubit=q[13])
    
    # T6: CZ gates (parallel for all three logical qubits)
    squin.cz(control=q[0], target=q[1])
    squin.cz(control=q[2], target=q[3])
    squin.cz(control=q[4], target=q[5])
    squin.cz(control=q[7], target=q[8])
    squin.cz(control=q[9], target=q[10])
    squin.cz(control=q[11], target=q[12])
    
    # T7: √Y on q[1,2,4], q[8,9,11], and q[15,16,18] (parallel)
    squin.ry(angle=1.5707963267948966, qubit=q[1])
    squin.ry(angle=1.5707963267948966, qubit=q[2])
    squin.ry(angle=1.5707963267948966, qubit=q[4])
    squin.ry(angle=1.5707963267948966, qubit=q[8])
    squin.ry(angle=1.5707963267948966, qubit=q[9])
    squin.ry(angle=1.5707963267948966, qubit=q[11])


    squin.cx(control=q[0], target=q[7])
    squin.cx(control=q[1], target=q[8])
    squin.cx(control=q[2], target=q[9])
    squin.cx(control=q[3], target=q[10])
    squin.cx(control=q[4], target=q[11])
    squin.cx(control=q[5], target=q[12])
    squin.cx(control=q[6], target=q[13])

    squin.measure(q[7])
    squin.measure(q[8])
    squin.measure(q[9])
    squin.measure(q[10])
    squin.measure(q[11])
    squin.measure(q[12])
    squin.measure(q[13])

    for i in range(7, 14):
        squin.reset(q[i])

    
    squin.ry(angle=-1.5707963267948966, qubit=q[7])
    squin.ry(angle=-1.5707963267948966, qubit=q[8])
    squin.ry(angle=-1.5707963267948966, qubit=q[9])
    squin.ry(angle=-1.5707963267948966, qubit=q[10])
    squin.ry(angle=-1.5707963267948966, qubit=q[11])
    squin.ry(angle=-1.5707963267948966, qubit=q[12])
    # T2: CZ gates (parallel for all three logical qubits)
    
    squin.cz(control=q[8], target=q[9])
    squin.cz(control=q[10], target=q[11])
    squin.cz(control=q[12], target=q[13])
    
    # T3: √Y on q[6], q[13], and q[20] (parallel)
    squin.ry(angle=1.5707963267948966, qubit=q[13])
    
    # T4: CZ gates (parallel for all three logical qubits)
    squin.cz(control=q[7], target=q[10])
    squin.cz(control=q[9], target=q[12])
    squin.cz(control=q[11], target=q[13])
    
    # T5: √Y on q[2,3,4,5,6], q[9,10,11,12,13], and q[16,17,18,19,20] (parallel)
    squin.ry(angle=1.5707963267948966, qubit=q[9])
    squin.ry(angle=1.5707963267948966, qubit=q[10])
    squin.ry(angle=1.5707963267948966, qubit=q[11])
    squin.ry(angle=1.5707963267948966, qubit=q[12])
    squin.ry(angle=1.5707963267948966, qubit=q[13])
    
    # T6: CZ gates (parallel for all three logical qubits)
    squin.cz(control=q[7], target=q[8])
    squin.cz(control=q[9], target=q[10])
    squin.cz(control=q[11], target=q[12])
    
    # T7: √Y on q[1,2,4], q[8,9,11], and q[15,16,18] (parallel)
    squin.ry(angle=1.5707963267948966, qubit=q[8])
    squin.ry(angle=1.5707963267948966, qubit=q[9])
    squin.ry(angle=1.5707963267948966, qubit=q[11])


    squin.cx(control=q[0], target=q[7])
    squin.cx(control=q[1], target=q[8])
    squin.cx(control=q[2], target=q[9])
    squin.cx(control=q[3], target=q[10])
    squin.cx(control=q[4], target=q[11])
    squin.cx(control=q[5], target=q[12])
    squin.cx(control=q[6], target=q[13])

    for i in range(7):
        squin.h(q[7 + i])

    squin.measure(q[7])
    squin.measure(q[8])
    squin.measure(q[9])
    squin.measure(q[10])
    squin.measure(q[11])
    squin.measure(q[12])
    squin.measure(q[13])

# Visualize with white background
from IPython.display import HTML, display
tsim_circ = bloqade.tsim.Circuit(_fixed)
diagram = tsim_circ.diagram(height=500)
# Get HTML and wrap with white background
html_str = diagram._repr_html_() if hasattr(diagram, '_repr_html_') else str(diagram)
display(HTML(f'<div style="background-color: white; padding: 20px; border: 1px solid #ddd; display: inline-block;">{html_str}</div>'))

In [58]:
from random import sample
import bloqade.stim

shot_length = 1000
stim_circ = bloqade.stim.Circuit(_fixed)
sampler = stim_circ.compile_sampler()
samples = sampler.sample(shots=shot_length)

def green_blue_orange_stabilizers(sample):
    z = (-1)**sample
    green = z[1] * z[2] * z[4] * z[5]
    blue = z[0] * z[1] * z[2] * z[3]
    orange = z[2] * z[3] * z[4] * z[6]
    return [green, blue, orange]

print((-1)**samples[0])
print((-1)**samples[1])
print((-1)**samples[2])

print(green_blue_orange_stabilizers(samples[0]))
print(green_blue_orange_stabilizers(samples[1]))
print(green_blue_orange_stabilizers(samples[2]))

first_stabilizer = green_blue_orange_stabilizers(samples[0])
    
for i, sample in enumerate(samples): 
    stabilizer = green_blue_orange_stabilizers(sample)
    if stabilizer != first_stabilizer:
        print(f"error at {i}")

len(samples)

[-1 -1  1  1  1 -1  1]
[ 1 -1 -1  1  1  1 -1]
[1 1 1 1 1 1 1]
[np.int64(1), np.int64(1), np.int64(1)]
[np.int64(1), np.int64(1), np.int64(1)]
[np.int64(1), np.int64(1), np.int64(1)]


1000

In [61]:
from random import sample
import bloqade.stim

shot_length = 1000
stim_circ = bloqade.stim.Circuit(_fixed)
sampler = stim_circ.compile_sampler()
samples = sampler.sample(shots=shot_length)

In [62]:
samples[0]

array([ True, False, False,  True,  True,  True, False,  True, False,
       False,  True,  True,  True, False])

In [ ]:


def green_blue_orange_stabilizers(sample):
    z = (-1)**sample
    green = z[1] * z[2] * z[4] * z[5]
    blue = z[0] * z[1] * z[2] * z[3]
    orange = z[2] * z[3] * z[4] * z[6]
    return [green, blue, orange]

print((-1)**samples[0])
print((-1)**samples[1])
print((-1)**samples[2])

print(green_blue_orange_stabilizers(samples[0]))
print(green_blue_orange_stabilizers(samples[1]))
print(green_blue_orange_stabilizers(samples[2]))

first_stabilizer = green_blue_orange_stabilizers(samples[0])
    
for i, sample in enumerate(samples): 
    stabilizer = green_blue_orange_stabilizers(sample)
    if stabilizer != first_stabilizer:
        print(f"error at {i}")

len(samples)

## Logical CNOT Circuit with Measurements and Feedforward

Based on the circuit diagram, you need:

1. **Two logical qubits** (14 qubits total: 7 per logical qubit)
   - Top logical qubit: `|ψ⟩_L` (arbitrary state)
   - Bottom logical qubit: `|+⟩_L` (logical plus state)

2. **Logical CNOT gate** (transversal CNOT between corresponding physical qubits)
   - Control: Top logical qubit (qubits 0-6)
   - Target: Bottom logical qubit (qubits 7-13)

3. **Measurements**:
   - Z-basis measurement on bottom logical qubit
   - X-basis measurement on bottom logical qubit

4. **Reset operation**: Re-initialize bottom logical qubit to `|0⟩_L` after first measurement

5. **Conditional gates (feedforward)**:
   - Z_i gate on top logical qubit, controlled by first measurement
   - X_i gate on top logical qubit, controlled by second measurement

6. **Logical state preparation**:
   - Prepare `|+⟩_L` on bottom logical qubit

In [18]:
# Logical CNOT with measurements and feedforward
# This implements the circuit from the figure:
# - Two logical qubits (14 physical qubits: 0-6 for top, 7-13 for bottom)
# - Logical CNOT (transversal CNOT)
# - Z and X measurements on bottom logical qubit
# - Conditional Z and X gates on top logical qubit based on measurements

@squin.kernel
def logical_cnot_with_feedforward():
    # Allocate 14 qubits: 7 for top logical qubit, 7 for bottom logical qubit
    q = squin.qalloc(14)
    
    # Top logical qubit: qubits 0-6 (|ψ⟩_L - already prepared by logical_qubit)
    # Bottom logical qubit: qubits 7-13 (|+⟩_L - need to prepare)
    
    # Prepare |+⟩_L on bottom logical qubit (qubits 7-13)
    # This is done by applying Hadamard to all physical qubits
    for i in range(7, 14):
        squin.h(q[i])
    
    # First CNOT: Top (control) → Bottom (target)
    # Transversal CNOT: CNOT between corresponding physical qubits
    for i in range(7):
        squin.cx(control=q[i], target=q[i+7])
    
    # Z-basis measurement on bottom logical qubit
    # Measure all physical qubits of bottom logical qubit in Z-basis
    for i in range(7, 14):
        squin.measure(q[i])  # Z-measurement (default)
    
    # Reset bottom logical qubit to |0⟩_L
    for i in range(7, 14):
        squin.reset(q[i])
    
    # Second CNOT: Top (control) → Bottom (target)
    for i in range(7):
        squin.cx(control=q[i], target=q[i+7])
    
    # X-basis measurement on bottom logical qubit
    # First apply H to measure in X-basis, then measure
    for i in range(7, 14):
        squin.h(q[i])
        squin.measure(q[i])
    
    # Conditional gates based on measurement outcomes
    # Note: In Bloqade, measurement results are available via Stim sampling
    # For feedforward, you would use Python if statements based on measurement results
    # This is a simplified version - actual feedforward requires post-processing
    
    # Conditional Z_i on top logical qubit (controlled by first measurement)
    # This would be: if measurement_result == 1: apply Z to top logical qubit
    # For now, we'll leave this as a placeholder - actual implementation needs
    # to sample measurements and conditionally apply gates
    
    # Conditional X_i on top logical qubit (controlled by second measurement)
    # Similar to above - requires measurement sampling and conditional logic

print("✓ Logical CNOT circuit with measurements defined")
print("Note: Feedforward (conditional gates) requires measurement sampling")
print("      and Python if statements based on measurement outcomes")

✓ Logical CNOT circuit with measurements defined
Note: Feedforward (conditional gates) requires measurement sampling
      and Python if statements based on measurement outcomes


In [19]:
# Fixed-parameter version for Tsim visualization
# Logical CNOT with measurements (14 qubits: 7 for each logical qubit)
@squin.kernel
def _fixed_logical_cnot():
    # Allocate 14 qubits: 7 for top logical qubit, 7 for bottom logical qubit
    q = squin.qalloc(14)
    
    # Prepare |+⟩_L on bottom logical qubit (qubits 7-13)
    squin.h(q[7])
    squin.h(q[8])
    squin.h(q[9])
    squin.h(q[10])
    squin.h(q[11])
    squin.h(q[12])
    squin.h(q[13])
    
    # First CNOT: Top (control) → Bottom (target)
    # Transversal CNOT: CNOT between corresponding physical qubits
    squin.cx(control=q[0], target=q[7])
    squin.cx(control=q[1], target=q[8])
    squin.cx(control=q[2], target=q[9])
    squin.cx(control=q[3], target=q[10])
    squin.cx(control=q[4], target=q[11])
    squin.cx(control=q[5], target=q[12])
    squin.cx(control=q[6], target=q[13])
    
    # Z-basis measurement on bottom logical qubit
    squin.measure(q[7])
    squin.measure(q[8])
    squin.measure(q[9])
    squin.measure(q[10])
    squin.measure(q[11])
    squin.measure(q[12])
    squin.measure(q[13])
    
    # Reset bottom logical qubit to |0⟩_L
    squin.reset(q[7])
    squin.reset(q[8])
    squin.reset(q[9])
    squin.reset(q[10])
    squin.reset(q[11])
    squin.reset(q[12])
    squin.reset(q[13])
    
    # Second CNOT: Top (control) → Bottom (target)
    squin.cx(control=q[0], target=q[7])
    squin.cx(control=q[1], target=q[8])
    squin.cx(control=q[2], target=q[9])
    squin.cx(control=q[3], target=q[10])
    squin.cx(control=q[4], target=q[11])
    squin.cx(control=q[5], target=q[12])
    squin.cx(control=q[6], target=q[13])
    
    # X-basis measurement on bottom logical qubit
    # First apply H to measure in X-basis, then measure
    squin.h(q[7])
    squin.measure(q[7])
    squin.h(q[8])
    squin.measure(q[8])
    squin.h(q[9])
    squin.measure(q[9])
    squin.h(q[10])
    squin.measure(q[10])
    squin.h(q[11])
    squin.measure(q[11])
    squin.h(q[12])
    squin.measure(q[12])
    squin.h(q[13])
    squin.measure(q[13])
    
    # Note: Conditional Z and X gates would be applied based on measurement results
    # This requires post-processing measurement outcomes, which is done outside the kernel

# Visualize with white background
from IPython.display import HTML, display
tsim_circ = bloqade.tsim.Circuit(_fixed_logical_cnot)
diagram = tsim_circ.diagram(height=600)
# Get HTML and wrap with white background
html_str = diagram._repr_html_() if hasattr(diagram, '_repr_html_') else str(diagram)
display(HTML(f'<div style="background-color: white; padding: 20px; border: 1px solid #ddd; display: inline-block;">{html_str}</div>'))

In [20]:
# Magic state injection circuit for [[7,1,3]] color code
# Exact gate sequence from the specification:
# T1: √Y† on q[0–5]
# T2: CNOT(1→{2,3,4,5})
# T3: CNOT(0→{2,4})
# T4: √Y on q[2,3,4,5]
# T5: CNOT(2→3), CNOT(4→5)
# T6: CNOT(6→{1,3,5})
# T7: √Y on q[1,2,4]

@squin.kernel
def logical_qubit(theta: float = 0.0, phi: float = 0.0):
    # Allocate 7 qubits
    q = squin.qalloc(7)
    
    # T1: √Y† on q[0–5]
    # √Y† = RY(-π/2) = -1.5707963267948966
    squin.ry(angle=-1.5707963267948966, qubit=q[0])
    squin.ry(angle=-1.5707963267948966, qubit=q[1])
    squin.ry(angle=-1.5707963267948966, qubit=q[2])
    squin.ry(angle=-1.5707963267948966, qubit=q[3])
    squin.ry(angle=-1.5707963267948966, qubit=q[4])
    squin.ry(angle=-1.5707963267948966, qubit=q[5])
    
    # T2: CZ(1→2), CZ(3→4), CZ(5→6)
    squin.cz(control=q[1], target=q[2])
    squin.cz(control=q[3], target=q[4])
    squin.cz(control=q[5], target=q[6])

    
    # T3: √Y on q[6]
    # √Y = RY(π/2) = 1.5707963267948966
    squin.ry(angle=1.5707963267948966, qubit=q[6])
    
    # T4: CZ(0→3), CZ(2→5), CZ(4→6)
    squin.cz(control=q[0], target=q[3])
    squin.cz(control=q[2], target=q[5])
    squin.cz(control=q[4], target=q[6])
    
    # T5: √Y on q[2,3,4,5,6]
    squin.ry(angle=1.5707963267948966, qubit=q[2])
    squin.ry(angle=1.5707963267948966, qubit=q[3])
    squin.ry(angle=1.5707963267948966, qubit=q[4])
    squin.ry(angle=1.5707963267948966, qubit=q[5])
    squin.ry(angle=1.5707963267948966, qubit=q[6])

    # T6: CZ(0→1), CZ(2→3), CZ(4→5)
    squin.cz(control=q[0], target=q[1])
    squin.cz(control=q[2], target=q[3])
    squin.cz(control=q[4], target=q[5])
    
    # T6: √Y on q[1,2,4]
    squin.ry(angle=1.5707963267948966, qubit=q[1])
    squin.ry(angle=1.5707963267948966, qubit=q[2])
    squin.ry(angle=1.5707963267948966, qubit=q[4])
    
    